In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from collections import Counter
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.utils import shuffle


In [38]:
test = pd.read_csv('../output_preprocessed/test_last_transaction.csv')
test_features = test[test.columns[2:]]

predictions = pd.DataFrame()
predictions['msno']=test['msno']

In [39]:
train = pd.read_csv('../output_preprocessed/train_last_transaction.csv')


In [41]:
train.is_churn

0          1
1          1
2          1
3          1
4          1
5          1
6          1
7          1
8          1
9          1
10         1
11         1
12         1
13         1
14         1
15         1
16         1
17         1
18         1
19         1
20         1
21         1
22         1
23         1
24         1
25         1
26         1
27         1
28         1
29         1
          ..
1963861    0
1963862    0
1963863    0
1963864    0
1963865    0
1963866    0
1963867    0
1963868    0
1963869    0
1963870    0
1963871    0
1963872    0
1963873    0
1963874    0
1963875    0
1963876    0
1963877    0
1963878    0
1963879    0
1963880    0
1963881    0
1963882    0
1963883    0
1963884    0
1963885    0
1963886    0
1963887    0
1963888    0
1963889    0
1963890    0
Name: is_churn, Length: 1963891, dtype: int64

In [44]:

desired_apriori=0.10

# Get the indices per target value
idx_0 = train[train.is_churn == 0].index
idx_1 = train[train.is_churn == 1].index



# Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target=0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
train = train.loc[idx_list].reset_index(drop=True)

In [45]:
print(len(train[train.is_churn == 1]))
print(len(train[train.is_churn == 0]))

150801
1357208


1: 150801
0: 1813090

In [46]:
train = train[train.columns[1:]]
train_features = train[train.columns[1:]]
train_labels = train["is_churn"]
del train

In [47]:
train_features.head()

,trans_count,logs_count,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,28,0.0,13.0,39.0,1.0,9.0,20130907.0,40.0,30.0,149.0,...,20170416.0,0.0,20161029.0,0.0,0.0,0.0,0.0,5.0,5.0,964.570
1,27,15.0,5.0,19.0,1.0,9.0,20100413.0,37.0,30.0,149.0,...,20170419.0,0.0,20170330.0,0.0,0.0,1.0,0.0,4.0,1.0,1185.043
2,29,11.0,1.0,0.0,0.0,7.0,20130807.0,41.0,30.0,99.0,...,20170425.0,0.0,20170331.0,1.0,0.0,0.0,0.0,0.0,1.0,21.991
3,7,21.0,1.0,0.0,0.0,7.0,20161001.0,41.0,30.0,99.0,...,20170430.0,0.0,20170331.0,1.0,0.0,0.0,1.0,26.0,28.0,5864.276
4,10,19.0,15.0,0.0,0.0,4.0,20160603.0,40.0,30.0,149.0,...,20170406.0,0.0,20170330.0,0.0,0.0,0.0,1.0,6.0,7.0,1834.225


In [48]:
(train_labels == 1).sum()/len(train_labels)

0.10000006631260158

In [49]:
train_features.columns

Index(['trans_count', 'logs_count', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')

In [50]:
def replacemean(x, t, mean):
    if(x < t):
        return mean
    else:
        return x

In [51]:
mean_registration_init_time = train_features[train_features["registration_init_time"] > 1]["registration_init_time"].mean()
train_features["registration_init_time"] = train_features["registration_init_time"].apply(lambda x: replacemean(x, 1, mean_registration_init_time))
test_features["registration_init_time"] = test_features["registration_init_time"].apply(lambda x: replacemean(x, 1, mean_registration_init_time))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
mean_transaction_date = train_features[train_features["transaction_date"] > 1]["transaction_date"].mean()
train_features["transaction_date"] = train_features["transaction_date"].apply(lambda x: replacemean(x, 1, mean_transaction_date))
test_features["transaction_date"] = test_features["transaction_date"].apply(lambda x: replacemean(x, 1, mean_transaction_date))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
mean_date = train_features[train_features["date"] > 1]["date"].mean()
train_features["date"] = train_features["date"].apply(lambda x: replacemean(x, 1, mean_date))
test_features["date"] = test_features["date"].apply(lambda x: replacemean(x, 1, mean_date))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
cols_to_transform = ["city", "gender", "payment_method_id"]

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train_features.loc[:][cols_to_transform])

ONE_HOT_train = (enc.transform(train_features.loc[:][cols_to_transform]).toarray()).transpose()
ONE_HOT_test = (enc.transform(test_features.loc[:][cols_to_transform]).toarray()).transpose()

for col in cols_to_transform:
    del train_features[col]
    del test_features[col]

for i in range(0, ONE_HOT_train.shape[0]):
    train_features["ONE_HOT_"+str(i)] = ONE_HOT_train[i]
    test_features["ONE_HOT_"+str(i)] = ONE_HOT_test[i]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm.fit(train_features)

train_features_preprocessed = norm.transform(train_features)
test_features_preprocessed = norm.transform(test_features)

In [56]:
def compute_logloss(pred_proba, ischurn):
    logloss = -((ischurn*np.log(pred_proba)).sum() + ((1 - ischurn)*np.log(1 - pred_proba)).sum())
    return (logloss / len(pred_proba))

In [57]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                
                print(gini_normalized(y_holdout,y_pred))
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

#         results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
#         print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)
        return res

In [60]:
from xgboost import XGBClassifier
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

import lightgbm as lgb


K = 5
# kf = KFold(n_splits=K)
kf = StratifiedKFold(n_splits=K)

#gbm = XGBClassifier(max_delta_step = 1)
#gbm.fit(train_features_preprocessed, train_labels)
#predictions["pred_gbm"] = gbm.predict_proba(test_features_preprocessed)[:,1]


# lgbm = lgb.LGBMClassifier(n_estimatorsobjective='binary')
# lgbm.fit(train_features_preprocessed, train_labels)
# predictions["pred_lgbm"] = lgbm.predict_proba(test_features_preprocessed)[:,1]



#adb = ensemble.AdaBoostClassifier()
#adb.fit(train_features_preprocessed, train_labels)
#predictions["pred_adb"] = adb.predict_proba(test_features_preprocessed)[:,1]


#lg = LogisticRegression()
#lg.fit(train_features_preprocessed, train_labels)
#predictions["pred_lg"] = lg.predict_proba(test_features_preprocessed)[:,1]



In [ ]:
#max_depth=6, subsample=0.8, colsample_bytree=0.8, reg_alpha =2, reg_lambda = 6,

for est in [1000]:
    for et in [0.005]:
        for depth in [-1]:
            for col in [0.6,0.8,0.9,1]:
                for sub in [0.6,0.8,0.9,1]:
                    print("///////////////////////////////////////////////////////////////////////////////")
                    lgbm = lgb.LGBMClassifier(n_estimators=est, learning_rate = et , max_depth=depth, subsamble = sub , colsample_bytree=col, objective='binary')
                    targets = np.asarray([0.]*test.shape[0])
                    for train_index,test_index in kf.split(train_features_preprocessed,train_labels):
                        X_train, y_train = train_features_preprocessed[train_index].copy(),train_labels[train_index].copy()
                        X_test, y_test = train_features_preprocessed[test_index].copy(),train_labels[test_index].copy()
                        fit_lgbm = lgbm.fit(X_train, y_train,eval_metric='')
                        y_predict=fit_lgbm.predict_proba(X_test)
                        score=compute_logloss(y_predict[:,1],y_test.values)
                        print(score)
                        targets += fit_lgbm.predict_proba(test_features_preprocessed)[:,1]
                        del y_test, X_train, X_test, y_train
                    targets /=K
                    test_target = pd.DataFrame()
                    test_target['msno']=test['msno']
                    test_target['is_churn']=targets
                    test_target.to_csv('../output/output_LGB_est'+str(est)+'_lr'+str(et)+'_maxdepth'+str(depth)+'_col'+str(col)+'_sub'+str(sub)+'.csv', index=False)

#predictions.to_csv("../output/from_kaggle_kernels_and_calculs.csv", float_format='%.6f', index = False)

///////////////////////////////////////////////////////////////////////////////
0.0843943028925


In [21]:
cl1 = lgb.LGBMClassifier(n_estimators=1000, learning_rate =0.005 , max_depth=-1,objective='binary')
lr1 = LogisticRegression()
stack = Ensemble(n_splits=5,
        stacker = lr1,
        base_models = (cl1))

targets = stack.fit_predict(X,y,test.drop(['id'],axis=1))[:,1]
test_target = pd.DataFrame()
test_target['id']=test['id']
test_target['target']=targets
# test_target.to_csv('./output/output_LGB_showoff.csv', index=False)


///////////////////////////////////////////////////////////////////////////////
est=1000 et=0.005 depth=-1
0.0832268324013
0.0840225550256
0.130625156578
0.136549348377
0.136210738066
///////////////////////////////////////////////////////////////////////////////
est=1000 et=0.005 depth=7
0.0831035400493
0.0839660360624
0.132396293088
0.138398769378
0.138883725537

